In [1]:
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.insert(0,'/home/nigam/librascal_cs/librascal/build/')

In [2]:
from ase.io import read, write
import ase
import json
from tqdm.notebook import tqdm
class tqdm_reusable:
    def __init__(self, *args, **kwargs):
        self._args = args
        self._kwargs = kwargs

    def __iter__(self):
        return tqdm(*self._args, **self._kwargs).__iter__()
    
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.utils import (get_radial_basis_covariance, get_radial_basis_pca, 
                          get_radial_basis_projections, get_optimal_radial_basis_hypers )
from rascal.utils import radial_basis
from rascal.utils import WignerDReal, ClebschGordanReal, spherical_expansion_reshape, lm_slice, real2complex_matrix, compute_lambda_soap

In [3]:
# a library of utilities and wrapper to compute pair features and manipulate a Hamiltonian-like target
from ncnice import * 

# Check Bispectrum

In [4]:
from ncnice.representations import compute_rho3i_lambda

In [5]:
spherical_expansion_hypers = {
    "interaction_cutoff": 4,
    "max_radial": 8,
    "max_angular": 6,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "GTO",
}

spex = SphericalExpansion(**spherical_expansion_hypers)
mycg = ClebschGordanReal(spherical_expansion_hypers["max_angular"])

In [6]:
frames = read('data/ethanol-structures.xyz', ':')
# frames = read('water/water_randomized_1000.xyz', ':')
# for f in frames:
#     f.cell=[100,100,100]
#     f.positions+=50
#     # use same specie for all atoms so we get a single projection matrix
#     f.numbers = f.numbers*0+1    

# spherical_expansion_hypers = get_optimal_radial_basis_hypers(spherical_expansion_hypers, frames, expanded_max_radial=12)


# # ... and we replicate it 
# pm = spherical_expansion_hypers['optimization']['RadialDimReduction']['projection_matrices'][1] 
# spherical_expansion_hypers['optimization']['RadialDimReduction']['projection_matrices'] = { i: pm for i in range(1000) }
spex = SphericalExpansion(**spherical_expansion_hypers)

In [7]:
frames = read('data/ethanol-structures.xyz', ':')
# frames = read('/water_randomized_1000.xyz', ':')
for f in frames:
    f.cell=[100,100,100]
    f.positions+=50
rhoi = compute_rhoi(frames[:1], spex, spherical_expansion_hypers)*1e0

In [8]:
rho2i_l, prho2i_l = compute_all_rho2i_lambda(rhoi, mycg, rho2i_pca=None)

In [9]:
rho3ilambda, prho3 = compute_rho3i_lambda(rho2i_l,rhoi, 0, mycg, prho2i_l)

In [10]:
#MANUAL computation of bispectrum from librascal/nice_demo.ipynb

selframe = frames[:1];         # frame used for the test
feat_scaling = 1e0         
feats = spex.transform(selframe).get_features(spex)
ref_feats = feat_scaling*spherical_expansion_reshape(feats, **spherical_expansion_hypers)

nice2_full = mycg.combine_nice(ref_feats, ref_feats)
bisp_nice = np.zeros(ref_feats.shape[:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) )
for l in range(spherical_expansion_hypers["max_angular"]+1):
    # while we are at it, we also reorder the indices in a bispectrum-like way
    bisp_nice[...,l] = mycg.combine_einsum(nice2_full[...,lm_slice(l)], 
                                         ref_feats[...,lm_slice(l)], 
                                         L=0, 
                                         combination_string="ianANlL,ibp->ianlANLbp" )[...,0]

In [11]:
bisp_nice[0,0,1,:,0,2,:,0,2,0]

array([[ 3.53478718e-07,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -1.71389200e-07,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.40871215e-07,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.04372720e-07,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  8.28421227e-08,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -2.14357119e-08,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.4668940

In [12]:
rho3ilambda[0,0,1,0,2,0,2,:,0]

array([ 3.53478718e-07, -1.71389200e-07,  1.40871215e-07, -2.04372720e-07,
        8.28421227e-08, -2.14357119e-08,  1.46689408e-08,  1.70524787e-24,
       -2.16170216e-09,  2.02957006e-09, -3.64477175e-10, -8.47299546e-08,
        1.00206736e-09,  7.10603360e-08, -1.63934345e-11, -1.10189128e-08,
       -2.26245530e-10,  4.37147981e-09, -2.16170216e-09,  8.52245753e-08,
        1.45952169e-08, -3.64477175e-10, -1.46168020e-08, -2.53861596e-08,
        1.00206736e-09,  9.30308771e-09,  1.35549074e-10, -1.63934345e-11,
        1.39403377e-08, -2.26245530e-10,  8.52245753e-08,  1.45952169e-08,
        5.63762664e-11,  2.91102939e-08, -1.46168020e-08, -2.53861596e-08,
       -5.54465397e-09,  9.30308771e-09,  1.35549074e-10,  1.39403377e-08,
        1.45952169e-08,  5.63762664e-11,  2.91102939e-08, -2.53861596e-08,
       -5.54465397e-09,  1.35549074e-10,  5.63762664e-11,  2.91102939e-08,
       -5.54465397e-09,  2.91102939e-08])

they are the **same** modulo noise. because we dont account for sorting l's in the manual computation and the storage format is inefficient, the shapes of the matrices are different

# rho2iP

In [9]:
def compute_rho1ijp_species(rho1ijp, frame, cg): 
    """groups neighbors j by species"""
    species = frame.numbers
    nspecies = len(set(species))
    shape = rho1ijp.shape[:2]+(nspecies,) +rho1ijp.shape[2:]
    rho1ijp_spec = np.zeros(shape)
    
    for ispe, spe in enumerate(sorted(set(species))):
        idx_spe = np.where(species==spe)[0]
#         print(spe, idx_spe)
        for j in range(rho1ijp.shape[1]):
            if j in idx_spe:
                rho1ijp_spec[:,j,ispe, ...]= rho1ijp[:,j,...]
    return rho1ijp_spec

In [10]:
hypers_ij = deepcopy(spherical_expansion_hypers)
hypers_ij["expansion_by_species_method"] = "structure wise"
spex_ij = SphericalExpansion(**hypers_ij)

frame = frames[0]
fgij = compute_gij(frame, spex_ij, hypers_ij)
rhoi = compute_rhoi(frame, spex, spherical_expansion_hypers)
rho1ijp, prhoijp = compute_rho1ijp_lambda(rhoi, fgij,0, mycg)
rho1ij, prhoij = compute_rho1ij_lambda(rhoi, fgij,0, mycg)

In [11]:
rho1ijp_l, prho1ijp_l = compute_all_rho1ijp_lambda(rhoi, fgij, mycg)

In [17]:
def compute_rho1ip_lambda(rhoi, rho1ijp_l, fgij, frame, L, cg, prho1ijp_l):
    """compute combined rhoi and rhoi-MP"""
    rho1ijp_spec = {}
    prhoijp_spec = {}
    rho1ip_lambda = {}
    lmax = int(np.sqrt(rhoi.shape[-1])) -1
    for l in range(lmax+1):
        rho1ijp_spec[l] = compute_rho1ijp_species(rho1ijp_l[l], frame, cg)
        rho1ip_lambda[l] = np.sum(rho1ijp_spec[l], axis=1)  # MP - atom density 
    
    # do the usual gig of angular coupling 
    nl=0
    for l1 in range(lmax+1):
        for l2 in range(lmax+1):
            if abs(l2 - l1) > L or l2 + l1 < L:
                continue
            nl += 1*rho1ip_lambda[l2].shape[4]
# multiplication above to account for (l1,l2,l3) combinations. rho1ip_lambda[L].shape[4] accounts for l1,l2
#  combinations of g(rij) and rhoj
    shape = rhoi.shape[:3] + rho1ip_lambda[L].shape[1:4] +(nl, 2*L+1,)
    rho_combined_lam = np.zeros(shape)
    prho_combined_lam = np.ones(nl, dtype = int)*(1-2*(L%2))
    
    il=0
    for l1 in range(lmax+1):
        for l2 in range(lmax+1):
            if abs(l2 - l1) > L or l2 + l1 < L:
                continue
            rho_combined_lam[...,il:il+rho1ip_lambda[l2].shape[4],:] = cg.combine_einsum(rhoi[...,lm_slice(l1)], rho1ip_lambda[l2],
                                            L, combination_string="ian,ibNMl->ianbNMl")

            prho_combined_lam[il:il+rho1ip_lambda[l2].shape[4]] = (1-2*(l1%2)) * prho1ijp_l[l2]
            il+=rho1ip_lambda[l2].shape[4]

    return rho_combined_lam, prho_combined_lam

In [22]:
rho_combined_lam, prho_combined_lam = compute_rho1ip_lambda(rhoi, rho1ijp_l, fgij, frame, 1, mycg, prho1ijp_l)

In [23]:
rho_combined_lam.shape

(9, 3, 8, 3, 24, 8, 483, 3)

# Compute bispectrum as sum over rho2ij lambda - NOT RIGHT

In [ ]:
hypers_ij = deepcopy(spherical_expansion_hypers)
# hypers_ij["expansion_by_species_method"] = "structure wise"
spex_ij = SphericalExpansion(**hypers_ij)
scale=1e3
for f in frames[:1]:
    fgij = compute_gij(f, spex_ij, hypers_ij)
    rhoi = compute_rhoi(f, spex, spherical_expansion_hypers)
    rho2i_l, prho2i_l = compute_all_rho2i_lambda(rhoi, mycg, rho2i_pca=None)
    rho2ij, prho2ij = compute_rho2ij_lambda(rho2i_l, fgij, 0, mycg, prho2i_l)
    

In [ ]:
shape = (rho2ij.shape[0],
         rhoi.shape[1],rho2ij.shape[2], 
         rho2ij.shape[3], rho2ij.shape[4], 
         rho2ij.shape[5], rho2ij.shape[6],
         rho2ij.shape[7], rho2ij.shape[8]
        )
bisp_gij = np.zeros(shape)
# parity = np.ones(nl, dtype = int)*(1-2*(L%2))

In [ ]:
rho2ij.shape

In [ ]:
for f in frames[:1]:
    fgij = compute_gij(f, spex_ij, hypers_ij)
    rhoi = compute_rhoi(f, spex, spherical_expansion_hypers)
    rho2i_l, prho2i_l = compute_all_rho2i_lambda(rhoi, mycg, rho2i_pca=None)
    rho2ij, prho2ij = compute_rho2ij_lambda(rho2i_l, fgij, 0, mycg, prho2i_l)
    shape = (rho2ij.shape[0],
         rhoi.shape[1],rho2ij.shape[2], 
         rho2ij.shape[3], rho2ij.shape[4], 
         rho2ij.shape[5], rho2ij.shape[6],
         rho2ij.shape[7], rho2ij.shape[8]
        )
    bisp_gij = np.zeros(shape)
#     parity = np.ones(nl, dtype = int)*(1-2*(L%2))
    species = f.numbers
    nspecies = rhoi.shape[1]
    for ispe, spe in enumerate(set(species)):
        idx_spe = np.where(species==spe)[0]
        print(spe, idx_spe)
        bisp_gij[:,ispe, ...]= np.sum(rho2ij[:,idx_spe,...],axis =1)

In [ ]:
bisp_gij.shape

In [ ]:
bisp_gij[0,0,1,0,2,0,2,:,0][np.where(bisp_gij[0,0,1,0,2,0,2,:,0])]

In [ ]:
rho3ilambda[0,0,1,0,2,0,2,:,0][np.where(rho3ilambda[0,0,1,0,2,0,2,:,0])]

In [ ]:
rho3ilambda[0][np.where(rho3ilambda[0])]

In [ ]:
bisp_gij[0][np.where(bisp_gij[0])]*2.14dd

In [ ]:
bisp_gij.shape

In [ ]:
def compute_mp(frames,hypers, lmax, nui, nuj, cg, rhoi_pca=None, rhoij_rho2i_pca=None,rho2i_pca=None, scale=1):
    spex = SphericalExpansion(**hypers)
    rhoi = compute_rhoi(frames, spex, hypers)

    # compresses further the spherical expansion features across species
    if rhoi_pca is not None:
        rhoi = apply_rhoi_pca(rhoi, rhoi_pca)

    # makes sure that the spex used for the pair terms uses adaptive species
    hypers_ij = deepcopy(hypers)
    hypers_ij["expansion_by_species_method"] = "structure wise"
    spex_ij = SphericalExpansion(**hypers_ij)

    tnat = 0
    els = list(orbs.keys())
    nel = len(els)
    # prepare storage
#     elL = list(itertools.product(els,range(lmax+1),[-1,1]))
#     hetL = [ (els[i1], els[i2], L, pi) for i1 in range(nel) for i2 in range((i1+1 if half_hete else 0), nel) for L in range(lmax+1) for pi in [-1,1] ]
#     feats = dict(diag = { L: [] for L in elL },
#                  offd_p = { L: [] for L in elL },
#                  offd_m = { L: [] for L in elL },
#                  hete =   { L: [] for L in hetL },)

    if rhoij_rho2i_pca is None and rho2i_pca is not None:
        rhoij_rho2i_pca = rho2i_pca

    #before = tracemalloc.take_snapshot()
    for f in frames:
        fnat = len(f.numbers)
        frhoi = rhoi[tnat:tnat+fnat]*scale
        fgij = compute_gij(f, spex_ij, hypers_ij)*scale
        for L in range(lmax+1):
            if nu==0:
                lrhonui, lprhonui = np.ones((fnat, 1, 2*L+1)), np.ones((1))
            elif nui==1 or nuj==1:
                lrhonui, lprhonui = compute_rho1i_lambda(frhoi, L, cg)
        
        rho_mp = 
        

In [ ]:
frames,hypers, lmax, nui, nuj, cg = frames, spherical_expansion_hypers, 2, 1, 1, mycg
rhoi_pca=None
rhoij_rho2i_pca=None
rho2i_pca=None
scale=1
tnat = 0
hypers_ij = deepcopy(hypers)
hypers_ij["expansion_by_species_method"] = "structure wise"
spex_ij = SphericalExpansion(**hypers_ij)
rhoi = compute_rhoi(frames[:1], spex, hypers)
# els = list(orbs.keys())
# nel = len(els)
for f in frames[:1]:
    fnat = len(f.numbers)
    frhoi = rhoi[tnat:tnat+fnat]*scale
    rhonui, prhonui = frhoi, None
    fgij = compute_gij(f, spex_ij, hypers_ij)*scale
    for L in range(lmax+1):
        #lrho0i, lprho0i = np.ones((fnat, 1, 2*L+1)), np.ones((1))
        lrho1i, lprho1i = compute_rho1i_lambda(frhoi, L, cg)
        
        lrho0ij, prho0ij = compute_rho0ij_lambda(rhonui, fgij, L, cg, prhonui)
        

In [ ]:
np.where(rhonui[:,0,:, lm_slice(2)]-lrho1i)

In [ ]:
lrho0ij

In [ ]:
?compute_rho0ij_lambda

In [ ]:
def compute_rho0ij_lambda(rhoi, gij, L, cg,  prfeats = None): # prfeats is (in analogy with rho2ijlambda) the parity, but is not really necessary)
    """ computes |rho^0_{ij}; lm> """
    rho0ij = gij[..., lm_slice(L)].reshape((gij.shape[0], gij.shape[1], -1, 2*L+1))
    return rho0ij, np.ones(rho0ij.shape[2])

def compute_rho1ij_lambda(rhoj, gij, L, cg, prfeats = None): 
    """ computes |rho^1_{ij}; lm> """

    lmax = int(np.sqrt(gij.shape[-1])) -1
    # can't work out analytically how many terms we have, so we precompute it here
    nl = 0
    for l1 in range(lmax + 1):
        for l2 in range(lmax + 1):  # |rho_i> and |rho_ij; g> are not symmetric so we need all l2
            if abs(l2 - l1) > L or l2 + l1 < L:
                continue
            nl += 1

    rhoj = rhoj.reshape((rhoj.shape[0], -1, rhoj.shape[-1]))
    # natom, natom, nel*nmax, nmax, lmax+1, lmax+1, M
    shape = (rhoj.shape[0], rhoj.shape[0],
             rhoj.shape[1] , gij.shape[2], nl, 2*L+1)
    rho1jilambda = np.zeros(shape)
    parity = np.ones(nl, dtype = int)*(1-2*(L%2))

    il = 0
    for l1 in range(lmax+1):
        for l2 in range(lmax+1):
            if abs(l2 - l1) > L or l2 + l1 < L:
                continue
            rho1jilambda[:,:,:,:,il] = cg.combine_einsum(rhoj[...,lm_slice(l1)], gij[...,lm_slice(l2)],
                                                L, combination_string="in,ijN->ijnN")
            parity[il] *= (1-2*(l1%2)) * (1-2*(l2%2))
            il+=1

    return rho1jilambda, parity

In [ ]:
spex = SphericalExpansion(**hypers)
rhoi = compute_rhoi(frames, spex, hypers)
hypers_ij = deepcopy(hypers)
hypers_ij["expansion_by_species_method"] = "structure wise"
spex_ij = SphericalExpansion(**hypers_ij)

# Test features and Hamiltonian transformation

In [ ]:
import re
from ase.data import atomic_numbers
import json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


In [ ]:
# frames = read('qm7/qm7b-chno.xyz', ':100')
# fock = np.load('qm7/qm7-chno-fock.npy', allow_pickle=True)
frames = read('water_random/water_randomized_1000.xyz',':')
for f in frames:
    f.pbc=False
    
focks = np.load('water/water-fock.npy', allow_pickle=True)
orbs = json.loads(json.load(open('water/orbs.json', "r")))

we have to fix the L=1 terms that are stored in a weird order. we do this as post-processing 
and one should undo the change if the matrix is to be read back into pyscf

In [ ]:
for i in range(len(focks)):
    focks[i] = pyscf_fix_l1(focks[i], frames[i], orbs)

## Rotational behavior of features

In [ ]:

from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.utils import (get_radial_basis_covariance, get_radial_basis_pca, 
                          get_radial_basis_projections, get_optimal_radial_basis_hypers )
from rascal.utils import radial_basis
from rascal.utils import (WignerDReal, ClebschGordanReal, 
                          spherical_expansion_reshape,
                          lm_slice, real2complex_matrix, xyz_to_spherical, spherical_to_xyz)
from rascal.utils.cg_utils import _r2c as r2c
from rascal.utils.cg_utils import _c2r as c2r
from rascal.utils.cg_utils import _cg as clebsch_gordan
from rascal.utils.cg_utils import _rotation as rotation
from rascal.utils.cg_utils import _wigner_d as wigner_d

### Rotation of features

In [ ]:
frame = frames[90]
frame.cell = [100,100,100]
frame.positions += 50
rotframe = frame.copy()
rotframe.positions = rotframe.positions @ mrot.T
rotframe.cell = rotframe.cell @ mrot.T   # rotate also the cell

feats = spex.transform(frame).get_features(spex)
feats *= 1e6
rfeats = spherical_expansion_reshape(feats, **spherical_expansion_hypers)

g= get_gij_fast(frame, spex,spherical_expansion_hypers)

In [ ]:
rotfeats = spex.transform(rotframe).get_features(spex)
rotfeats *= 1e6
rotfeats = spherical_expansion_reshape(rotfeats, **spherical_expansion_hypers)

rotg = get_gij_fast(rotframe, spex,spherical_expansion_hypers)

Check rotation of $| \overline{\rho_{ij}^0; \lambda \mu} \rangle \equiv \langle n | g; \lambda \mu\rangle $

In [ ]:
np.linalg.norm(WD.rotate(mk_rho0ijlambda_fast(g, 4, mycg)) - mk_rho0ijlambda_fast(rotg, 4, mycg))/np.linalg.norm(mk_rho0ijlambda_fast(g, 4, mycg))

Check rotation of $| \overline{\rho_{i}^1; \lambda \mu} \rangle $

In [ ]:
(np.linalg.norm(WD.rotate(rfeats[...,lm_slice(3)]) - rotfeats[...,lm_slice(3)]))/np.linalg.norm(rfeats[...,lm_slice(3)])

Check $| \overline{\rho_{i}^2; \lambda \mu} \rangle $

In [ ]:
lsoap=mk_rho2ilambda_fast(rfeats, 1, mycg)

In [ ]:
rotlsoap =  mk_rho2ilambda_fast(rotfeats, 1, mycg)
np.linalg.norm(WD.rotate(lsoap) -rotlsoap)/np.linalg.norm(lsoap)

Check rotation of $| \overline{\rho_{ij}^1; 00} \rangle $

In [ ]:
rhoij = mk_rho1ij_fast(rfeats, g, mycg)
rhoij_rot = mk_rho1ij_fast(rotfeats, rotg, mycg)

In [ ]:
np.linalg.norm(WD.rotate(rhoij[...,np.newaxis]) - rhoij_rot[...,np.newaxis])/np.linalg.norm(rhoij)

Check rotation of $| \overline{\rho_{ij}^1; \lambda \mu} \rangle $

In [ ]:
rhoijlm= mk_rho1ijlambda_fast(rfeats, g, 3, mycg)
rhoijlm_rot = mk_rho1ijlambda_fast(rotfeats, rotg, 3, mycg)
np.linalg.norm(WD.rotate(rhoijlm) - rhoijlm_rot)/np.linalg.norm(rhoijlm)

# Simple regression test

In [ ]:
iwater = 99
fock = np.load('water/water-fock.npy', allow_pickle=True)[iwater]
orbs = json.loads(json.load(open("water/orbs.json", "r")))

there has to be a model for each (n1,l1,n2,l2,L) entry in the coupled representation of the Fock matrix

In [ ]:
frame = read('water/water_coords_1000.xyz',':')[iwater]
frame.cell = [100,100,100]
frame.positions += 50
frame.symbols

In [ ]:
fock = pyscf_fix_l1(fock, frame, orbs)
fock_blocks = pyscf_to_blocks(fock, frame, orbs)
fock_blocks_c = to_coupled(fock_blocks, mycg)

In [ ]:
ffeats = do_full_features([frame], orbs, spherical_expansion_hypers, 4, mycg, scale=1e3)

In [ ]:
FR = FockRegression(orbs, alpha=1e-18, solver='svd')

In [ ]:
FR.fit(ffeats, fock_blocks_c)

In [ ]:
fpred = FR.predict(ffeats)

In [ ]:
fpred['diag'][(2,1,2,1)]

In [ ]:
fock_blocks_c['diag'][(2,1,2,1)]

In [ ]:
unfock = blocks_to_pyscf(to_decoupled(fpred, mycg), frame, orbs)

In [ ]:
plt.matshow((unfock-fock).astype(float))
np.linalg.norm(unfock-fock) # bingo!

In [ ]:
np.mean(np.abs(np.linalg.eigvalsh(fock.astype(float))-np.linalg.eigvalsh(unfock)))

In [ ]:
plt.plot(np.linalg.eigvalsh(fock.astype(float)), 'b.')
plt.plot(np.linalg.eigvalsh(unfock.astype(float)), 'r--')

#### Rotation and permutation 

In [ ]:
iwater = 99
frame = read('water/water_coords_1000.xyz',':')[iwater]
frame.cell = [100,100,100]
frame.positions += 50
print(frame.symbols)
fock = np.load('water/water-fock.npy', allow_pickle=True)[iwater]
orbs = json.loads(json.load(open("water/orbs.json", "r")))

In [ ]:
fock = pyscf_fix_l1(fock, frame, orbs)
fock_blocks = pyscf_to_blocks(fock, frame, orbs)
fock_blocks_c = to_coupled(fock_blocks, mycg)
feats = do_full_features([frame], orbs, spherical_expansion_hypers, 4, mycg, scale=1e3)

In [ ]:
FR = FockRegression(orbs, alpha=1e-18, solver='svd')
FR.fit(feats, fock_blocks_c)
fpred = FR.predict(feats)

In [ ]:
fock_original = blocks_to_pyscf(to_decoupled(fpred, mycg), frame, orbs)

In [ ]:
frame_rotperm = frame.copy()
iperm = np.arange(len(frame.numbers), dtype=int)
np.random.shuffle(iperm)
frame_rotperm.numbers = frame_rotperm.numbers[iperm]
frame_rotperm.positions = frame_rotperm.positions[iperm]
print(frame_rotperm.symbols)

abc = np.random.uniform(size=(3))*np.pi
WD = WignerDReal(spherical_expansion_hypers["max_angular"], *abc)
WD.rotate_frame(frame_rotperm)

In [ ]:
feat_rotperm = do_full_features([frame_rotperm], orbs, spherical_expansion_hypers, 4, mycg, scale=1e3)

In [ ]:
pred_rotperm = FR.predict(feat_rotperm)

In [ ]:
fock_rotperm = blocks_to_pyscf(to_decoupled(pred_rotperm, mycg), frame_rotperm, orbs)

In [ ]:
plt.matshow((fock_rotperm-fock_original).astype(float))
np.linalg.norm(fock_rotperm-fock_original) 

In [ ]:
plt.plot(np.linalg.eigvalsh(fock_original.astype(float)), 'b.')
plt.plot(np.linalg.eigvalsh(fock_rotperm.astype(float)), 'r--') 
print(np.mean(np.abs(np.linalg.eigvalsh(fock_original)-np.linalg.eigvalsh(fock_rotperm))))